## Procedural Notes  

### Enzyme/Pathway Selection  
Kegg glycolysis/Glucogenesis-Reference Pathway:  

>Enzymes selected for each pathway are:  
>- Glycolysis: K00844, K11645, K01689, K16370  
>- Citric Acid Cycle: K01647, K00030, K01900, K00116  
>- Pentose Phosphate Pathway: K13937, K00036, K01807, K06859  



In [1]:
# Creating the genes, pathways and enzymes table with respective assigned columns
import sqlite3
conn = sqlite3.connect('my.db')
c = conn.cursor()
# to avoid conflict with previously built table
c.execute("""DROP TABLE genes""")
c.execute("""DROP TABLE pathways""")
c.execute("""DROP TABLE enzymes""")

# Caution: use INTEGER instead of INT for primary key generation
c.execute("""CREATE TABLE genes (id INTEGER PRIMARY KEY, name TEXT, description TEXT, organism TEXT, nucleotide_sequence TEXT,
chromosome TEXT, location TEXT, start INT, end INT, strand VARCHAR(1), translated_sequence TEXT);""")

c.execute("""CREATE TABLE pathways (id INTEGER PRIMARY KEY, name TEXT, description TEXT);""")

c.execute("""CREATE TABLE enzymes (id INTEGER PRIMARY KEY, kegg_id TEXT, name TEXT, description TEXT, function TEXT, ec_number TEXT);""")

conn.commit()


### Inserting values into the pathway table
1. get the names of the pathways
2. search up the description of each pathway
3. Take in two list with equal length and insert the into the corresponding field

In [2]:
import sqlite3
conn = sqlite3.connect('my.db')
c = conn.cursor()
cmd = ''' INSERT INTO pathways (name, description) VALUES(?,?)'''

names = ["Glycolysis","Citric Acid Cycle","Pentose Phosphate"]
descriptions = ["the breakdown of glucose by enzymes, releasing energy and pyruvic acid." , "the sequence of reactions by which most living cells generate energy during the process of aerobic respiration. It takes place in the mitochondria, consuming oxygen, producing carbon dioxide and water as waste products, and converting ADP to energy-rich ATP","a metabolic pathway parallel to glycolysis, It generates NADPH and pentoses (5-carbon sugars) as well as ribose 5-phosphate, the last one a precursor for the synthesis of nucleotides"]


# Take in two list with equal length and insert the into the corresponding field
for i in range(len(names)):
    name = names[i]
    description = descriptions[i]
    temp_value = (name,description)
    c.execute(cmd,temp_value)
    conn.commit()


### Query the name and ec_number of the selected Enzymes from KEGG
1. Make a sample.list file in the current directory with the following lines  
K00844   
K11645   
K01689  
K16370  
K01647  
K00030  
K01900  
K00116  
K13937  
K00036  
K01807  
K06859  

2. In the terminal execute: `curl -g -s -S http://rest.kegg.jp/list/ko | grep -f sample.list | sed "s/cpd\://" > sample_extracted.table.txt`

3. Inspect the output file sample_extracted.table.txt  

In [3]:
# parse in the sample_extracted.table.txt for enzymes table insertion
def parse_extracted_table():
    filename = "sample_extracted.table.txt"
    in_file = open(filename, "rt")
    lines = []
    # separate each line in the for for inspection and downstream analysis
    for line in in_file:
        lines.append(line.rstrip('\n'))
        print(line)
    return lines

lines = parse_extracted_table()


# the get balues function split each line in the sample_extracted.table.txt to render the kegg_id, 
# enzyme name, function, and ec_number as separate lists
def get_values(lines):
    kegg_ids = []
    names = []
    functions = []
    ec_numbers = []
    for line in lines:
        # first, separate out the ec_number by the "[" symbol
        line_ec = line.split("[")
        ec_number = line_ec[-1]
        # separate the by the ";" symbol, the first half will be kegg_id and the name, the second part function
        line_2_part = line_ec[0].split(";")
        # line_part1 contains kegg_id and the name
        line_part1 = line_2_part[0].split()
        # line_part2 contains function description
        line_part2 = line_2_part[1].split()
#         print(line_part1)
#         print(line_part2)
        kegg_id = line_part1[0]
        # join the 1:end of part1 to form the name
        temp_name = line_part1[1:]
        temp_name[:] = [''.join(temp_name[:])]
        name = temp_name[0]
        # join all segments in part2 to form the function string
        temp_function = line_part2[:]
        temp_function[:] = [" ".join(temp_function[:])]
        function = temp_function[0]     
        kegg_ids.append(kegg_id)
        names.append(name)
        functions.append(function)
        ec_numbers.append(ec_number)
    return kegg_ids,names,functions,ec_numbers


# inspect the 3 output lists
get_values(lines)





ko:K00030	IDH3; isocitrate dehydrogenase (NAD+) [EC:1.1.1.41]

ko:K00036	G6PD, zwf; glucose-6-phosphate 1-dehydrogenase [EC:1.1.1.49 1.1.1.363]

ko:K00116	mqo; malate dehydrogenase (quinone) [EC:1.1.5.4]

ko:K00844	HK; hexokinase [EC:2.7.1.1]

ko:K01647	CS, gltA; citrate synthase [EC:2.3.3.1]

ko:K01689	ENO, eno; enolase [EC:4.2.1.11]

ko:K01807	rpiA; ribose 5-phosphate isomerase A [EC:5.3.1.6]

ko:K01900	LSC2; succinyl-CoA synthetase beta subunit [EC:6.2.1.4 6.2.1.5]

ko:K06859	pgi1; glucose-6-phosphate isomerase, archaeal [EC:5.3.1.9]

ko:K11645	fbaB; fructose-bisphosphate aldolase, class I [EC:4.1.2.13]

ko:K13937	H6PD; hexose-6-phosphate dehydrogenase [EC:1.1.1.47 3.1.1.31]

ko:K16370	pfkB; 6-phosphofructokinase 2 [EC:2.7.1.11]



(['ko:K00030',
  'ko:K00036',
  'ko:K00116',
  'ko:K00844',
  'ko:K01647',
  'ko:K01689',
  'ko:K01807',
  'ko:K01900',
  'ko:K06859',
  'ko:K11645',
  'ko:K13937',
  'ko:K16370'],
 ['IDH3',
  'G6PD,zwf',
  'mqo',
  'HK',
  'CS,gltA',
  'ENO,eno',
  'rpiA',
  'LSC2',
  'pgi1',
  'fbaB',
  'H6PD',
  'pfkB'],
 ['isocitrate dehydrogenase (NAD+)',
  'glucose-6-phosphate 1-dehydrogenase',
  'malate dehydrogenase (quinone)',
  'hexokinase',
  'citrate synthase',
  'enolase',
  'ribose 5-phosphate isomerase A',
  'succinyl-CoA synthetase beta subunit',
  'glucose-6-phosphate isomerase, archaeal',
  'fructose-bisphosphate aldolase, class I',
  'hexose-6-phosphate dehydrogenase',
  '6-phosphofructokinase 2'],
 ['EC:1.1.1.41]',
  'EC:1.1.1.49 1.1.1.363]',
  'EC:1.1.5.4]',
  'EC:2.7.1.1]',
  'EC:2.3.3.1]',
  'EC:4.2.1.11]',
  'EC:5.3.1.6]',
  'EC:6.2.1.4 6.2.1.5]',
  'EC:5.3.1.9]',
  'EC:4.1.2.13]',
  'EC:1.1.1.47 3.1.1.31]',
  'EC:2.7.1.11]'])

In [4]:
# insert values into the enzymes table with 4 equal length list names, functions, and kegg_ids and ec_numbers 
# into the corresponding columns of the enzymes table
import sqlite3
conn = sqlite3.connect('my.db')
c = conn.cursor()
cmd = ''' INSERT INTO enzymes (kegg_id, name, description, function, ec_number) VALUES(?,?,?,?,?)'''

# get the four lists
kegg_ids,names,functions,ec_numbers = get_values(lines)
descriptions = ['isocitrate dehydrogenase', 'glucose-6-phosphate 1-dehydrogenase', 'malate dehydrogenase', 'hexokinase', 'citrate synthase', 'enolase', 'ribose 5-phosphate isomerase', 'succinyl-CoA synthetase', 'glucose-6-phosphate isomerase', 'fructose-bisphosphate aldolase', 'hexose-6-phosphate dehydrogenase', '6-phosphofructokinase']
functions = ["Glycolysis"]*4+["Citric Acid Cycle"]*4 + ["Pentose Phosphate"]*4
# insertion
for i in range(len(names)):
    name = names[i]
    function = functions[i]
    description = descriptions[i]
    kegg_id = kegg_ids[i]
    ec_number = ec_numbers[i]
    temp_value = (kegg_id, name, description, function, ec_number)
    c.execute(cmd,temp_value)
    conn.commit()

In [5]:
from Bio import Entrez
from Bio import SeqIO
#Entrez.esearch(db,term, sort)
#Entrez.efetch(db, id, rettype, retmode)
#for enzymes and genes only; Entrez doesn't have pathway info


# a helper function to search and fetch for gene, nucleotide, and protein by specifying the db_type and the term
def search_fetch(db_type,term):
    Entrez.email = 'ych323@berkeley.edu'
    handle = Entrez.esearch(db = db_type,
                       term = term,
                       sort= 'relevance',
                       idtype= 'acc')
    fetched_dict = Entrez.read(handle)
#     print(fetched_dict)
    if not fetched_dict["IdList"]:
        data = "None"
        return data
    handle = Entrez.efetch(db = db_type, id = fetched_dict["IdList"][0], rettype = 'fasta', retmode = 'text')
    data = handle.read()
    return data


# build a term list of all the search "terms species[ORGN] enzyme_name"
# build the 3 * 12 entries of name, organism, and description of the gene
species_list = ["Drosophila","Escherichia coli","Homo sapiens"]
ter_list = [str(species) + "[ORGN]" + " " for species in species_list ]
term_list = [str(term) + str(description) for term in ter_list for description in descriptions]
name_list = [str(name) for term in ter_list for name in names]
organism_list = [str(species) for species in species_list for name in names]
descriptions_list = [str(description) for species in species_list for description in descriptions]

# for the strand column, the DNA nucleotide sequence stored in GeneBank are all 5' to 3' of the sense strand, and so the 
# sign of the strand should all be "+"
strand_list = ['+']*len(term_list)

# inspect the built lists 
print("term_list")
print(term_list)
print("name_list")
print(name_list)
print("organism_list")
print(organism_list)
print("descriptions_list")
print(descriptions_list)


term_list
['Drosophila[ORGN] isocitrate dehydrogenase', 'Drosophila[ORGN] glucose-6-phosphate 1-dehydrogenase', 'Drosophila[ORGN] malate dehydrogenase', 'Drosophila[ORGN] hexokinase', 'Drosophila[ORGN] citrate synthase', 'Drosophila[ORGN] enolase', 'Drosophila[ORGN] ribose 5-phosphate isomerase', 'Drosophila[ORGN] succinyl-CoA synthetase', 'Drosophila[ORGN] glucose-6-phosphate isomerase', 'Drosophila[ORGN] fructose-bisphosphate aldolase', 'Drosophila[ORGN] hexose-6-phosphate dehydrogenase', 'Drosophila[ORGN] 6-phosphofructokinase', 'Escherichia coli[ORGN] isocitrate dehydrogenase', 'Escherichia coli[ORGN] glucose-6-phosphate 1-dehydrogenase', 'Escherichia coli[ORGN] malate dehydrogenase', 'Escherichia coli[ORGN] hexokinase', 'Escherichia coli[ORGN] citrate synthase', 'Escherichia coli[ORGN] enolase', 'Escherichia coli[ORGN] ribose 5-phosphate isomerase', 'Escherichia coli[ORGN] succinyl-CoA synthetase', 'Escherichia coli[ORGN] glucose-6-phosphate isomerase', 'Escherichia coli[ORGN] fru

In [6]:
# Get the nucleotide sequences of each enzyme through searching entrez and fetch the most relevent result
def get_nucleotide_sequence(term_list):
    nucleotide_list = []
    for term in term_list:
        print(term)
        data = search_fetch("nucleotide",term)
        # If data is "None", then the entrez search has fetched no id and the idlist is empty, and 
        # so cannot be parsed by SeqIO, so should be directly append to the nucleotide list
        if data == "None":
            nucleotide_list.append(data)
            continue
            
        # If data is not "None", then the search_and_fetch function return the most relevent sequence, wrtie data into a
        # fasta file and then use SeqIO.parse to get the sequence to a string for sqlite insertion
        f = open('temp.fa','w')
        f.write(data)
        f.close()
        print("parsed")
        for seq in SeqIO.parse("temp.fa", "fasta"):
            nucleotide_list.append(str(seq.seq))
    return nucleotide_list

nucleotide_list = get_nucleotide_sequence(term_list)
# inspect first three entries of the nucleotide_list

print("peak nucleotide_list")
print(nucleotide_list[1:4])


Drosophila[ORGN] isocitrate dehydrogenase
parsed
Drosophila[ORGN] glucose-6-phosphate 1-dehydrogenase
parsed
Drosophila[ORGN] malate dehydrogenase
parsed
Drosophila[ORGN] hexokinase
parsed
Drosophila[ORGN] citrate synthase
parsed
Drosophila[ORGN] enolase
parsed
Drosophila[ORGN] ribose 5-phosphate isomerase
parsed
Drosophila[ORGN] succinyl-CoA synthetase
parsed
Drosophila[ORGN] glucose-6-phosphate isomerase
parsed
Drosophila[ORGN] fructose-bisphosphate aldolase
parsed
Drosophila[ORGN] hexose-6-phosphate dehydrogenase
Drosophila[ORGN] 6-phosphofructokinase
parsed
Escherichia coli[ORGN] isocitrate dehydrogenase
parsed
Escherichia coli[ORGN] glucose-6-phosphate 1-dehydrogenase
parsed
Escherichia coli[ORGN] malate dehydrogenase
parsed
Escherichia coli[ORGN] hexokinase
parsed
Escherichia coli[ORGN] citrate synthase
parsed
Escherichia coli[ORGN] enolase
parsed
Escherichia coli[ORGN] ribose 5-phosphate isomerase
parsed
Escherichia coli[ORGN] succinyl-CoA synthetase
parsed
Escherichia coli[ORGN

In [7]:
from Bio import Entrez
from Bio import SeqIO

# following similar approach as get_nucleotide to get the protein sequence from entrez search and fetch
def get_protein(term_list):
    protein_list = []
    for term in term_list:
        print(term)
        data = search_fetch("protein",term)
        if data == "None":
            protein_list.append(data)
            continue
        f = open('temp.fa','w')
        f.write(data)
        f.close()
        print("parsed")
        for seq in SeqIO.parse("temp.fa", "fasta"):
            protein_list.append(str(seq.seq))
    return protein_list
protein_list = get_protein(term_list)

#inspect the output protein list
print("peek protein_list")
print(protein_list[1:3])


Drosophila[ORGN] isocitrate dehydrogenase
parsed
Drosophila[ORGN] glucose-6-phosphate 1-dehydrogenase
parsed
Drosophila[ORGN] malate dehydrogenase
parsed
Drosophila[ORGN] hexokinase
parsed
Drosophila[ORGN] citrate synthase
parsed
Drosophila[ORGN] enolase
parsed
Drosophila[ORGN] ribose 5-phosphate isomerase
parsed
Drosophila[ORGN] succinyl-CoA synthetase
parsed
Drosophila[ORGN] glucose-6-phosphate isomerase
parsed
Drosophila[ORGN] fructose-bisphosphate aldolase
parsed
Drosophila[ORGN] hexose-6-phosphate dehydrogenase
Drosophila[ORGN] 6-phosphofructokinase
parsed
Escherichia coli[ORGN] isocitrate dehydrogenase
parsed
Escherichia coli[ORGN] glucose-6-phosphate 1-dehydrogenase
parsed
Escherichia coli[ORGN] malate dehydrogenase
parsed
Escherichia coli[ORGN] hexokinase
parsed
Escherichia coli[ORGN] citrate synthase
parsed
Escherichia coli[ORGN] enolase
parsed
Escherichia coli[ORGN] ribose 5-phosphate isomerase
parsed
Escherichia coli[ORGN] succinyl-CoA synthetase
parsed
Escherichia coli[ORGN

In [8]:
# to get the chromosome, start, and end position and possibly other information from the gene entrez search
# CAUTION: hasn't implemented through iteration yet
from Bio import Entrez
# function to separate lines of the gene search and fetch output txt file
def parse_gene_entrez():
    filename = "temp_gene_entrez.txt"
    in_file = open(filename, "rt")
    lines = []
    # separate each line in the for for inspection and downstream analysis
    for line in in_file:
        lines.append(line.rstrip('\n'))
    return lines

def find_loci(lines):
    locus = "not_in_entrez"
    for line in lines:
        if "Chromosome:" in line:
            locus = line
            break
    return locus

def find_annotation(lines):
    annotation = "not_in_entrez"
    for line in lines:
        if "Annotation:" in line:
            annotation = line
            break
    return annotation

def get_location(term_list):   
    loci = []
    annotations = []
    Entrez.email = 'ych323@berkeley.edu'
    for term in term_list:
        data = search_fetch("gene",term)
        if data == "None":
            locus = "None"
            annotation = "None"
        else:
            f = open('temp_gene_entrez.txt','w')
            f.write(data)
            f.close()
            lines = parse_gene_entrez()
            #print(lines)
            # Location and annotation are in the 5 and 6 of the lines list 
            # if there's no desired field, then append "insufficient info"
            locus = find_loci(lines)
            annotation = find_annotation(lines)
        loci.append(locus)
        annotations.append(annotation)
    return loci, annotations

loci, annotations = get_location(term_list)

print(loci)
print(annotations)


['Chromosome: 3L; Location: 3-24 cM', 'Chromosome: Un', 'Chromosome: X', 'Chromosome: X; Location: 1-28 cM', 'Chromosome: X', 'Chromosome: 4; Location: 4- cM', 'Chromosome: Un', 'Chromosome: 3L; Location: 3-9 cM', 'Chromosome: Un', 'Chromosome: Un', 'None', 'Chromosome: Un', 'not_in_entrez', 'not_in_entrez', 'not_in_entrez', 'not_in_entrez', 'not_in_entrez', 'not_in_entrez', 'not_in_entrez', 'not_in_entrez', 'not_in_entrez', 'not_in_entrez', 'None', 'not_in_entrez', 'Chromosome: 2; Location: 2q34', 'Chromosome: 1; Location: 1p36.22', 'Chromosome: 7; Location: 7q11.23', 'Chromosome: 10; Location: 10q22.1', 'Chromosome: 12; Location: 12q13.3', 'Chromosome: 12; Location: 12p13.31', 'Chromosome: 2; Location: 2p11.2', 'Chromosome: 13; Location: 13q14.2', 'Chromosome: 19; Location: 19q13.11', 'Chromosome: 16; Location: 16p11.2', 'Chromosome: 1; Location: 1p36.22', 'Chromosome: 12; Location: 12q13.11']
['Annotation: Chromosome 3L NT_037436.4 (8356404..8360906, complement)', 'not_in_entrez', '

In [9]:
# parse the chromosome and location from loci list and the start and end position from the annotations site
def parse_chromosome(loci):
    chromosomes = []
    locations = []
    for temp_string in loci: 
        if "Chromosome:" not in temp_string:
            location = temp_string
            chromosome = temp_string
        elif "Un" in temp_string:
            location = "unknown"
            chromosome = "unknown"
        else:
            #print(temp_string)
            temp_list = temp_string.split(";")
            chromosome = temp_list[0].split(":")[1]
            try:
                location = temp_list[1].split(":")[1]
            except:
                location = 'not in entrez'
        locations.append(location)
        chromosomes.append(chromosome)
    return chromosomes, locations

chromosomes, locations =  parse_chromosome(loci)

# to account that E.coli has no chromosome:
chromosomes[12:24] = ['circular prokaryotic DNA']*12
locations[12:24] = ['circular prokaryotic DNA']*12
print("chromosomes:")
print(chromosomes)
print("locations:")
print(locations)
# from the list we can see that at lot of genes does not have info in all organisms


chromosomes:
[' 3L', 'unknown', ' X', ' X', ' X', ' 4', 'unknown', ' 3L', 'unknown', 'unknown', 'None', 'unknown', 'circular prokaryotic DNA', 'circular prokaryotic DNA', 'circular prokaryotic DNA', 'circular prokaryotic DNA', 'circular prokaryotic DNA', 'circular prokaryotic DNA', 'circular prokaryotic DNA', 'circular prokaryotic DNA', 'circular prokaryotic DNA', 'circular prokaryotic DNA', 'circular prokaryotic DNA', 'circular prokaryotic DNA', ' 2', ' 1', ' 7', ' 10', ' 12', ' 12', ' 2', ' 13', ' 19', ' 16', ' 1', ' 12']
locations:
[' 3-24 cM', 'unknown', 'not in entrez', ' 1-28 cM', 'not in entrez', ' 4- cM', 'unknown', ' 3-9 cM', 'unknown', 'unknown', 'None', 'unknown', 'circular prokaryotic DNA', 'circular prokaryotic DNA', 'circular prokaryotic DNA', 'circular prokaryotic DNA', 'circular prokaryotic DNA', 'circular prokaryotic DNA', 'circular prokaryotic DNA', 'circular prokaryotic DNA', 'circular prokaryotic DNA', 'circular prokaryotic DNA', 'circular prokaryotic DNA', 'circula

In [10]:
def parse_annotation(annotations):
    starts = []
    ends = []
    for temp_string in annotations: 
        if "Annotation:" not in temp_string:
            start = temp_string
            end = temp_string
        elif "Un" in temp_string:
            start = temp_string
            end = temp_string
        else:
            #print(temp_string)
            a = temp_string
            b = a.split("(")
            c = b[-1].split(")")
            d = c[0].split(" ")
            start_end = d[0].split("..")
            start = start_end[0]
            end = start_end[1].split(",")[0]
        starts.append(start)
        ends.append(end)
    return starts, ends

starts, ends = parse_annotation(annotations)
print("starts:")
print(starts)
print("ends:")
print(ends)

starts:
['8356404', 'not_in_entrez', 'not_in_entrez', '9585675', 'not_in_entrez', 'not_in_entrez', 'not_in_entrez', '3814211', 'not_in_entrez', 'not_in_entrez', 'None', 'not_in_entrez', '1195123', '1934839', '3383330', '410144', '753185', '57883', '4313350', '764180', '4289297', '3070165', 'None', '4107552', '208236227', '9234767', '76048019', '69269991', '56271699', '6914450', '88691658', '47942656', '34353330', '30053090', '9234767', '48105253']
ends:
['8360906', 'not_in_entrez', 'not_in_entrez', '9589813', 'not_in_entrez', 'not_in_entrez', 'not_in_entrez', '3818383', 'not_in_entrez', 'not_in_entrez', 'None', 'not_in_entrez', '1196373', '1936314', '3384268', '411052', '754468', '58233', '4313799', '765049', '4290946', '3071244', 'None', '4108514', '208255143', '9271337', '76067508', '69401882', '56300391', '6923696', '88750935', '48001326', '34402413', '30070420', '9271337', '48146404']


### sanity check on the entrez gene output
> as we have all the lists for the gene table insertion, check that the length of the lists are consistent

- id INTEGER PRIMARY KEY  
- name TEXT  
- description TEXT  
- organism TEXT  
- nucleotide_sequence TEXT  
- chromosome TEXT  
- location TEXT  
- start INT  
- end INT  
- strand VARCHAR(1)  
- translated_sequence TEXT  

In [11]:
from Bio import Entrez
from Bio import SeqIO

print(len(name_list))
print(len(organism_list))
print(len(descriptions_list))
print(len(strand_list))
print(len(nucleotide_list))
print(len(protein_list))
print(len(chromosomes))
print(len(locations))
print(len(starts))
print(len(ends))
# all the list are in same order with the same length

# manually fix the None fields in cds and aa sequence the H6PD gene
h6pd_nt = str([s.seq for s in SeqIO.parse("H6PD_nt.fa", "fasta")][0])
h6pd_aa = str([s.seq for s in SeqIO.parse("H6PD_aa.fa", "fasta")][0])   
for i in range(len(name_list)):
    if nucleotide_list[i] == 'None':
        nucleotide_list[i] = h6pd_nt
        protein_list[i] = h6pd_aa

36
36
36
36
36
36
36
36
36
36


In [12]:
import sqlite3
conn = sqlite3.connect('my.db')
c = conn.cursor()
cmd = ''' INSERT INTO genes (name,description,organism, nucleotide_sequence,chromosome,location,start,end,strand,translated_sequence) VALUES(?,?,?,?,?,?,?,?,?,?)'''

# insertion
for i in range(len(name_list)):
    name = name_list[i]
    description = descriptions_list[i]
    organism = organism_list[i]
    nucleotide_sequence = nucleotide_list[i]
    chromosome = chromosomes[i]
    location = locations[i]
    start = starts[i]
    end = ends[i]
    strand = strand_list[i]
    translated_sequence = protein_list[i]
    temp_value = (name,description,organism, nucleotide_sequence,chromosome,location,start,end,strand,translated_sequence)
    c.execute(cmd,temp_value)
    conn.commit()

### Demo of the functionality of three individual tables
  


In [13]:
import pandas as pd
import sqlite3
conn = sqlite3.connect("my.db")

print("inspect the pathways table")
df1 = pd.read_sql_query("select * from pathways;", conn)
df1

inspect the pathways table


,id,name,description
0,1,Glycolysis,"the breakdown of glucose by enzymes, releasing..."
1,2,Citric Acid Cycle,the sequence of reactions by which most living...
2,3,Pentose Phosphate,"a metabolic pathway parallel to glycolysis, It..."


In [14]:
print("inspect the enzymes table")
df2 = pd.read_sql_query("select * from enzymes;", conn)
df2

inspect the enzymes table


,id,kegg_id,name,description,function,ec_number
0,1,ko:K00030,IDH3,isocitrate dehydrogenase,Glycolysis,EC:1.1.1.41]
1,2,ko:K00036,"G6PD,zwf",glucose-6-phosphate 1-dehydrogenase,Glycolysis,EC:1.1.1.49 1.1.1.363]
2,3,ko:K00116,mqo,malate dehydrogenase,Glycolysis,EC:1.1.5.4]
3,4,ko:K00844,HK,hexokinase,Glycolysis,EC:2.7.1.1]
4,5,ko:K01647,"CS,gltA",citrate synthase,Citric Acid Cycle,EC:2.3.3.1]
5,6,ko:K01689,"ENO,eno",enolase,Citric Acid Cycle,EC:4.2.1.11]
6,7,ko:K01807,rpiA,ribose 5-phosphate isomerase,Citric Acid Cycle,EC:5.3.1.6]
7,8,ko:K01900,LSC2,succinyl-CoA synthetase,Citric Acid Cycle,EC:6.2.1.4 6.2.1.5]
8,9,ko:K06859,pgi1,glucose-6-phosphate isomerase,Pentose Phosphate,EC:5.3.1.9]
9,10,ko:K11645,fbaB,fructose-bisphosphate aldolase,Pentose Phosphate,EC:4.1.2.13]


### Design of Associative tables
1. enzymes_in_pathways: a 2 columns table that link the enzyme id to pathway id  
2. genes_in_enzyme: a 2 columns table that link the gene id to enzyme id
2. genes_in_pathway: a 2 columns table that link the gene id to pathway id

In [15]:
print("inspect the first four genes in table with human ")
df3 = pd.read_sql_query("select name,description,chromosome,start,end,strand from genes where organism = 'Homo sapiens' limit 4", conn)
df3

inspect the first four genes in table with human 


,name,description,chromosome,start,end,strand
0,IDH3,isocitrate dehydrogenase,2,208236227,208255143,+
1,"G6PD,zwf",glucose-6-phosphate 1-dehydrogenase,1,9234767,9271337,+
2,mqo,malate dehydrogenase,7,76048019,76067508,+
3,HK,hexokinase,10,69269991,69401882,+


In [16]:
print("inspect the first four genes in genes table with E.coli ")
df4 = pd.read_sql_query("select name,description,chromosome,start,end,strand from genes where organism = 'Escherichia coli' limit 4", conn)
df4

inspect the first four genes in genes table with E.coli 


,name,description,chromosome,start,end,strand
0,IDH3,isocitrate dehydrogenase,circular prokaryotic DNA,1195123,1196373,+
1,"G6PD,zwf",glucose-6-phosphate 1-dehydrogenase,circular prokaryotic DNA,1934839,1936314,+
2,mqo,malate dehydrogenase,circular prokaryotic DNA,3383330,3384268,+
3,HK,hexokinase,circular prokaryotic DNA,410144,411052,+


In [17]:
print("inspect the first three genes in genes table with Drosophila ")
df5 = pd.read_sql_query("select name,description,chromosome,start,end,strand from genes where organism = 'Drosophila' and chromosome != 'None' and start != 'not_in_entrez' limit 5", conn)
df5

inspect the first three genes in genes table with Drosophila 


,name,description,chromosome,start,end,strand
0,IDH3,isocitrate dehydrogenase,3L,8356404,8360906,+
1,HK,hexokinase,X,9585675,9589813,+
2,LSC2,succinyl-CoA synthetase,3L,3814211,3818383,+


In [18]:
print("inspect the nucleotide sequence")
df6 = pd.read_sql_query("select id, name, nucleotide_sequence from genes limit 5", conn)
df6

inspect the nucleotide sequence


,id,name,nucleotide_sequence
0,1,IDH3,ACGCCGATTGCGCAACAATTTGACGATTGTACGCCACCCAAGAGAG...
1,2,"G6PD,zwf",CCAAAATTTCTTTGTTTCTATTGGGCAGTCAAAATTTTAAATTTTA...
2,3,mqo,AAAAAGTAGTGACTGGAAAATAATTGTGAAAAGTAACAGGTTCCCA...
3,4,HK,ATGCCATTTGTGGACCCCTCAGCGTCGCACATATACACGCCATATC...
4,5,"CS,gltA",GAATTCGTCAGAAATGAGCTAAACAAATTTAAATCATTAAATGCGA...


In [19]:
print("inspect the translated sequence")
df7 = pd.read_sql_query("select id, name, translated_sequence from genes limit 5", conn)
df7

inspect the translated sequence


,id,name,translated_sequence
0,1,IDH3,MFALRRTAAMMTSVHRQPHLSQAVFRANFAISAADPKTHKISMDNE...
1,2,"G6PD,zwf",MLPIDPNNEVAYSFVVFGASGRLALSRIFPALWQLYRDNRLPQATK...
2,3,mqo,MAEPIRVVVTGAAGQIAYSLLYMIARGEVFGKDQPIVLHLLDIPPM...
3,4,HK,MVHELCQQLLLTDEQVQELCYRILHELRRGLAKDTHPKANVKCFVT...
4,5,"CS,gltA",MSLYRISARKLSEAQKLPNVGAYVRMIAADGKSLRDVLAAKVPQEQ...


In [20]:
import sqlite3
conn = sqlite3.connect('my.db')
c = conn.cursor()
# to avoid conflict with previously built table
c.execute("""DROP TABLE enzymes_in_pathways""")
c.execute("""DROP TABLE genes_in_enzymes""")
c.execute("""DROP TABLE genes_in_pathways""")
# Caution: use INTEGER instead of INT for primary key generation
c.execute("""CREATE TABLE enzymes_in_pathways (pathway_id INT, enzyme_id INT);""")
c.execute("""CREATE TABLE genes_in_enzymes (enzyme_id INT, gene_id INT);""")
c.execute("""CREATE TABLE genes_in_pathways (pathway_id INT, gene_id INT);""")
conn.commit()

cmd = ''' INSERT INTO enzymes_in_pathways (pathway_id,enzyme_id) VALUES(?,?)'''
c = conn.cursor()
# build enzymes_in_pathways
pathways = [1]*4 + [2]*4 + [3]*4 
enzymes = list(range(1,13))
for i in range(len(pathways)):
    temp_val = (pathways[i],enzymes[i])
    c.execute(cmd,temp_val)
    conn.commit()

# build genes_in_enzymes
cmd = ''' INSERT INTO genes_in_enzymes (enzyme_id,gene_id) VALUES(?,?)'''
enzymes = list(range(1,13))*3
genes = list(range(1,37))
for i in range(len(genes)):
    temp_val = (enzymes[i],genes[i])
    c.execute(cmd,temp_val)
    conn.commit()

# build genes_in_enzymes
cmd = ''' INSERT INTO genes_in_pathways (pathway_id,gene_id) VALUES(?,?)'''
pathways = [1,1,1,1,2,2,2,2,3,3,3,3]*3
genes = list(range(1,37))
for i in range(len(genes)):
    temp_val = (pathways[i],genes[i])
    c.execute(cmd,temp_val)
    conn.commit()
    


### Demo of the use of associative tables
- through associative tables, we can select a group of more specific items by grouping them with a more general item

In [21]:
print("select enzyme by a specific pathway: all enzyme in Glycolysis pathway")
df8 = pd.read_sql_query("select * from enzymes where id in (select enzyme_id from enzymes_in_pathways where pathway_id = 1)", conn)
df8

select enzyme by a specific pathway: all enzyme in Glycolysis pathway


,id,kegg_id,name,description,function,ec_number
0,1,ko:K00030,IDH3,isocitrate dehydrogenase,Glycolysis,EC:1.1.1.41]
1,2,ko:K00036,"G6PD,zwf",glucose-6-phosphate 1-dehydrogenase,Glycolysis,EC:1.1.1.49 1.1.1.363]
2,3,ko:K00116,mqo,malate dehydrogenase,Glycolysis,EC:1.1.5.4]
3,4,ko:K00844,HK,hexokinase,Glycolysis,EC:2.7.1.1]


In [22]:
print("select genes by a specific pathway: all human enzymes in Glycolysis pathway")
df9 = pd.read_sql_query("select name,description,chromosome,start,end,strand from genes where organism = 'Homo sapiens' and id in (select gene_id from genes_in_pathways where pathway_id = 1)", conn)
df9

select genes by a specific pathway: all human enzymes in Glycolysis pathway


,name,description,chromosome,start,end,strand
0,IDH3,isocitrate dehydrogenase,2,208236227,208255143,+
1,"G6PD,zwf",glucose-6-phosphate 1-dehydrogenase,1,9234767,9271337,+
2,mqo,malate dehydrogenase,7,76048019,76067508,+
3,HK,hexokinase,10,69269991,69401882,+


In [23]:
print("select genes by a specific enzyme: genes from different species for the HK enzyme")
df10 = pd.read_sql_query("select * from genes where id in (select gene_id from genes_in_enzymes where enzyme_id = 4)", conn)
df10

select genes by a specific enzyme: genes from different species for the HK enzyme


,id,name,description,organism,nucleotide_sequence,chromosome,location,start,end,strand,translated_sequence
0,4,HK,hexokinase,Drosophila,ATGCCATTTGTGGACCCCTCAGCGTCGCACATATACACGCCATATC...,X,1-28 cM,9585675,9589813,+,MVHELCQQLLLTDEQVQELCYRILHELRRGLAKDTHPKANVKCFVT...
1,16,HK,hexokinase,Escherichia coli,AACACTACCAATAAGTTGGAGTCATTACCCGCGGAATGTAAAATTT...,circular prokaryotic DNA,circular prokaryotic DNA,410144,411052,+,XGSSHHHHHHGSTKYALVGDVGGTNARLALCDIASGEISQAKTYSG...
2,28,HK,hexokinase,Homo sapiens,CCCTTCTAACCTCACCAGGCTGTTATCAGGAGAAATGAAGCAATAA...,10,10q22.1,69269991,69401882,+,MIAAQLLAYYFTELKDDQVKK
